<a href="https://colab.research.google.com/github/sangttruong/IncomeVis/blob/master/VizIncIneq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Technical Appendix for Visualizing Income Inequality in the United States**

[Sang Truong](mailto:sangtruong_2021@depauw.edu) and [Prof. Humberto Barreto](mailto:hbarreto@depauw.edu)

Department of Economics and Management, DePauw University, Greencastle, IN 46135


## **Section 1. Introduction**

This notebook supports Barreto and Truong's 2020 paper, "Visualizing Income Inequality in the United States." It gives a detailed explanation of the transformation of the raw data to the 3D visualization. First, we import `pandas` for data manipulation and `numpy` for scientific computing. We mount Google Drive to Google Colab to read source files directly from Google Drive.

In [ ]:
import pandas as pd
import numpy as np
import json
from collections import OrderedDict
from google.colab import drive
drive.mount('/content/gdrive')
input_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/input/'
out_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/output/'

# Display 1 decimal
pd.options.display.float_format = '{:,.1f}'.format
# Show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Data for analysis are from [IPUMS-CPS](https://cps.ipums.org/cps/). We download all available years in one data extract from CPS. Documentation on downloading and variables can be found on the IPUMS website [1]. For IPUMS-CPS, all selected samples are ASEC. Although IPUMS CPS has data back to 1962, geographic location (STATEFIP) is only available since 1977: "STATEFIP is comparable for 1963-1967 and 1977 onward, years in which each state and the District of Columbia were separately identified. In the remaining years, two or more states share the same code, and these groupings change over time. In 1962, 8 states cannot be separately identified. In 1968-1972, 32 states cannot be separately identified, and in 1973-1976, 38 states cannot be separately identified. In these years, up to 5 states share the same code."

We download ASEC samples from 1977 to 2019 as csv in a zip file, extract (~600MB), rename as "ipums-cps.csv" and place it in an accessible Google drive location.

For updating, note that ASEC comes out in IPUMS in October.

Years are confusing. We have ASEC CPS from 1977 to 2019. The CPI99 and HHINCOME variables in a sample year is for the previous year. So, the 1977 sample has Consumr Price Index and Household Income for 1976. Thus, we compute RHHINCOME from sample year 1977 data that is actually real household income for 1976.

Variables list: CPI99 STATEFIP HHINCOME PERNUM RELATE AGE SEX RACE HISPAN EDUC (only the first three are actually needed -- we thought we could do visualizations of subgroups, but sample sizes are too small).

"The Census Bureau fielded the CPS 2014 ASEC sample using an experimental redesign. All respondents received new health insurance questions, but 3/8ths of the total sample was randomly selected to receive the redesigned income questions. The larger portion of the sample (5/8) was given the existing questions on income. The redesign attempted to address income under-reporting, in particular, retirement, pensions, annuities, and government cash-transfer programs. More accurate income reporting in turn allows for better measurement of poverty statistics." HFLAG = 0 indicates 5/8 sample, and HFLAG = 1 indicate 3/8 sample in 2014. For the 2014 to be compatible with other data year, we select HFLAG = 0. [3]

We also create labels for states and establish starting color scheme for the visualization.

In [ ]:
# Import raw data
raw = pd.read_csv(input_path + 'ipums-cps.csv')
raw = raw[raw.YEAR >= 1977] # This line should be here regardless if you have the data before 1977 or not. It won't hurt if you don't have the data.

# Import rpp and merge with raw 
rpp = pd.read_csv('gdrive/My Drive/Colab Notebooks/USIncomeVis/input/rpp.csv')
raw = pd.merge(raw, rpp, how='outer', on = ['YEAR', 'STATEFIP'])

# Select data with HFLAG != 1 and then drop HFLAG
raw = raw[raw.HFLAG !=1]
raw = raw.drop(columns = ['HFLAG'])
print(raw.describe())

# States labels and color scheme
statefip = list(set(raw.STATEFIP))
state_name = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
              'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
              'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
              'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
              'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
              'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
              'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
              'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
              'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
              'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

colors = ['#FF0000', '#FF0A00', '#FF1400', '#FF1E00', '#FF2800', '#FF3300', '#FF3D00',
          '#FF4700', '#FF5100', '#FF5B00', '#FF6600', '#FF7000', '#FF7A00', '#FF8400',
          '#FF8E00', '#FF9900', '#FFA300', '#FFAD00', '#FFB700', '#FFC100', '#FFCC00',
          '#FFD600', '#FFE000', '#FFEA00', '#FFF400', '#FFFF00', '#F4FF00', '#EAFF00',
          '#E0FF00', '#D6FF00', '#CCFF00', '#C1FF00', '#B7FF00', '#ADFF00', '#A3FF00',
          '#99FF00', '#8EFF00', '#84FF00', '#7AFF00', '#70FF00', '#66FF00', '#5BFF00',
          '#51FF00', '#47FF00', '#3DFF00', '#32FF00', '#28FF00', '#1EFF00', '#14FF00',
          '#0AFF00', '#00FF00']
colors = pd.DataFrame(colors, columns = ['Color'], index = statefip)

             YEAR      SERIAL       MONTH                CPSID    ASECFLAG  \
count 7,477,891.0 7,477,891.0 7,477,891.0          5,526,971.0 7,477,891.0   
mean      1,999.0    42,834.1         3.0 14,609,170,480,628.5         1.0   
std          12.4    25,731.5         0.0  8,898,738,244,648.0         0.0   
min       1,977.0         1.0         3.0                  0.0         1.0   
25%       1,988.0    20,960.0         3.0                  0.0         1.0   
50%       2,001.0    41,657.0         3.0 19,970,300,219,500.0         1.0   
75%       2,010.0    62,644.0         3.0 20,080,101,688,900.0         1.0   
max       2,019.0    99,986.0         3.0 20,190,307,098,800.0         1.0   

          ASECWTH       CPI99    STATEFIP    HHINCOME      PERNUM  \
count 7,477,891.0 7,477,891.0 7,477,891.0 7,477,891.0 7,477,891.0   
mean      1,529.2         1.2        28.0    58,051.0         2.3   
std         911.3         0.6        15.7    67,951.9         1.4   
min           0.0    

## **Section 2. adjustIncome() Function**

In this analysis, we employ 3 deflators to make better comparisons: 1) Consumer Price Index (CPI) to create real dollar values of household income over time (RHHINCOME), 2) Household size (HHSIZE) to adjust for the number of people in a household (ERHHINCOME), and 3) State price

1) Because of inflation, to compare incomes over time, we need to convert nominal dollar values of HHINCOME to real values. We adjust incomes so they are all in 2018 dollars. 

2) We want to compare household incomes, but a household with two people with a total income of $100,000 is better off than a six-person household with the same income. We could just divide by the number of people in the household, but that ignores the fact that some expenses don't scale up linearly (a two-bedroom apartment is not double the rent of a one-bedroom and two people could share a car, for example). An *equivalence scale* adjusts household size. There are several options available. We tried the [*OECD equivalence scale*](http://www.oecd.org/els/soc/OECD-Note-EquivalenceScales.pdf) and the *square root equivalence scale*, as explained below.

3) 

Income adjustment can be done via `adjustIncome()` function. Below is its syntax and default parameters.

```
DataFrame adjustIncome(input_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/input/',
                       output_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/output/',
                       ipumcps_filename = 'ipums-cps.csv',
                       rpp_filename = 'rpp.csv')
```

In [ ]:
#@title **adjustIncome() Implementation for Developer**
from contextlib import contextmanager
import os, sys
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try: yield
        finally: sys.stdout = old_stdout
        
import pandas as pd
import numpy as np
import json
from collections import OrderedDict
from google.colab import drive
with suppress_stdout(): drive.mount('/content/gdrive')

# Display 1 decimal
pd.options.display.float_format = '{:,.1f}'.format
# Show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def adjustIncome(input_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/input/',
                 output_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/input/',
                 ipumcps_filename = 'ipums-cps.csv',
                 rpp_filename = 'rpp.csv'):
  # Import raw data
  raw = pd.read_csv(input_path + ipumcps_filename)
  raw = raw[raw.YEAR >= 1977] # This line should be here regardless if you have the data before 1977 or not. It won't hurt if you don't have the data.

  # Import rpp and merge with raw 
  rpp = pd.read_csv(input_path + rpp_filename)
  raw = pd.merge(raw, rpp, how='outer', on = ['YEAR', 'STATEFIP'])

  # Select data with HFLAG != 1 and then drop HFLAG
  raw = raw[raw.HFLAG !=1]
  raw = raw.drop(columns = ['HFLAG'])

  # Generate Real HHINCOME in 2018 dollars
  raw['RHHINCOME'] = raw['HHINCOME']*raw['CPI99']*1.507

  # Generate size
  raw['ISIZE'] = np.nan
  raw.loc[raw['PERNUM'] == 1, 'ISIZE'] = 1
  raw.loc[(raw['PERNUM'] != 1) & (raw['AGE'] > 16), 'ISIZE'] = 1
  raw.loc[(raw['PERNUM'] != 1) & (raw['AGE'] <= 16), 'ISIZE'] = 1

  # Generate household ID
  raw['HHID'] = np.nan
  length = sum(raw.loc[raw['PERNUM'] == 1, 'PERNUM'])
  raw.loc[raw['PERNUM'] == 1, 'HHID'] = np.arange(length)
  raw = raw.fillna(method='pad')

  # Generate effective size and merge with raw
  hhsize = raw.groupby(['HHID'])['ISIZE'].agg('sum').reset_index()
  raw = pd.merge(raw, hhsize, on = ['HHID'])
  raw = raw.rename(columns={'ISIZE_x': 'ISIZE', 'ISIZE_y': 'HHSIZE'})

  # Eliminate observations that has PERNUM != 1
  raw = raw[raw['PERNUM'] == 1]

  # Squared root of HHSIZE
  raw['HHSIZE'] = (raw['HHSIZE'])**(1/2)

  # Generate Real HHINCOME in 2018 dollars
  raw['ERHHINCOME'] = raw['RHHINCOME']/raw['HHSIZE']

  # Adjusting for state price using RPP
  raw['RPPRHHINCOME'] = raw['RHHINCOME']/(raw['RPP']/100)
  raw['RPPERHHINCOME'] = raw['ERHHINCOME']/(raw['RPP']/100)

  # Generate cumulative weight and percentage
  raw['CUMWTH'] = np.nan
  raw['PERCENTH'] = np.nan

  raw.to_csv(output_path + 'processed' + ipumcps_filename, index=False)
  return raw

In [ ]:
processedRaw = adjustIncome()

### **2.1. Consumer Price Index**

CPI reported by IPUMS-CPS is based on 1999 dollars. We convert CPI99 to 2018 dollars by multiplying by 1.507. Remember, CPI in a given sample year is actually the CPI for the previous year.

[This web page](https://cps.ipums.org/cps/cpi99.shtml) gives more explanation.
[HHINCOME](https://cps.ipums.org/cps-action/variables/HHINCOME#description_section) reports the total money income during the **previous calendar** year of all adult household members. "The amount should equal the sum of all household members' individual incomes as recorded in the IPUMS-CPS variable [INCTOT](https://cps.ipums.org/cps-action/variables/INCTOT#description_section). The persons included were those present in the household at the time of the survey. People who lived in the household during the previous year but were not still living there at the time of the survey are not included; household members who lived elsewhere during the previous year but had joined the household at the time of the survey are included."

HHINCOME includes sources of income like wages, salaries, and business income. It can be negative. Some of the component income sources (like [INCWAGE](https://cps.ipums.org/cps-action/variables/INCWAGE#description_section) have "disclosure avoidance measures" for individuals with high incomes. Our income range is from the 5th to the 95th percentile so we avoid negative values and problems with correctly measuring extremely high incomes.

In [ ]:
# Generate Real HHINCOME in 2018 dollars
raw['RHHINCOME'] = raw['HHINCOME']*raw['CPI99']*1.507
print(raw['RHHINCOME'].describe())

count   7,477,891.0
mean       85,836.7
std        81,624.5
min       -78,600.3
25%        36,943.6
50%        68,268.7
75%       110,405.1
max     3,451,328.3
Name: RHHINCOME, dtype: float64


### **2.2. Household Size SQRT**

Since our unit of analysis is household, we initialize a new dataframe (hhsize, or household size) and a new variable (ISIZE, or individual size) to account for the different in household size. For household $j$ in the sample with $n$ members,

$$ HHSIZE_j = \sum_{i = 1}^{n} ISIZE_{n} $$

For the OECD equivalence scale, each member of the household contributes to household size depending on age, like this:
* Household head (PERNUM = 1): ISIZE = 1 unit.
* Adult (PERNUM $\neq$ 1 and AGE > 16): ISIZE = 0.7 unit.
* Child (PERNUM $\neq$ 1 and AGE $\leq$ 16): ISIZE = 0.5 unit.

For the square root equivalence scale, everyone counts the same. In other words:
* Household head (PERNUM = 1): ISIZE = 1 unit.
* Adult (PERNUM $\neq$ 1 and AGE > 16): ISIZE = 1 unit.
* Child (PERNUM $\neq$ 1 and AGE $\leq$ 16): ISIZE = 1 unit.

But the equivalized household size is the square root of the sum of the members in the household. Johnson, et al. 2005, p. 13 explain the logic of the square root scale:

"This particular scale is
given by the square root of family size and indicates that the
resources for a two-person family must be 41 percent more [sqrt(2) is roughly 1.41]
than that of a single-person family for the two families to have
an equivalent standard of living. In general, the constant
elasticity scales are given by 
$$ family size^e $$
in which *e* is the
scale elasticity. Notice that if the elasticity equals one, then
the scale equals family size; there are no assumed economies
of scale in living arrangements and the equivalent resources
are simply the per capita resources. Alternatively, if the
elasticity equals zero, then there is no adjustment for family
size, there are complete economies of scale in living and the
marginal cost of another person is zero. Our chosen elasticity
of 0.5 lies halfway between these two implausible extremes
and results in “equivalent” consumer unit resources."

We found little difference between the two scales in this analysis. We think household income per equivalized person is better than just household income, but the specific equivalence scale is applied does not matter. [4]

To calculate household size computationally efficiently (i.e., to compute household size without having to iterate through nearly 8 million lines of data), we create a new variable called HHID (household ID) by assigning a unique number to each household. With HHID, we can group each household easily.

We then compute the household size for each household ID by taking the sum of all individual sizes in the household. After that, we merge HHSIZE with the raw dataset and drop PERNUM since we no longer need it. 

Now we can adjust RHHINCOME by HHSIZE to get *equivalized real household income*, ERHHINCOME. 

This is real household income per equivalent person.

In [ ]:
# Generate size
raw['ISIZE'] = np.nan
raw.loc[raw['PERNUM'] == 1, 'ISIZE'] = 1
raw.loc[(raw['PERNUM'] != 1) & (raw['AGE'] > 16), 'ISIZE'] = 1
raw.loc[(raw['PERNUM'] != 1) & (raw['AGE'] <= 16), 'ISIZE'] = 1

# Generate household ID
raw['HHID'] = np.nan
length = sum(raw.loc[raw['PERNUM'] == 1, 'PERNUM'])
raw.loc[raw['PERNUM'] == 1, 'HHID'] = np.arange(length)
raw = raw.fillna(method='pad')

# Generate effective size and merge with raw
hhsize = raw.groupby(['HHID'])['ISIZE'].agg('sum').reset_index()
raw = pd.merge(raw, hhsize, on = ['HHID'])
raw = raw.rename(columns={'ISIZE_x': 'ISIZE', 'ISIZE_y': 'HHSIZE'})

# Eliminate observations that has PERNUM != 1
raw = raw[raw['PERNUM'] == 1]

# Squared root of HHSIZE
raw['HHSIZE'] = (raw['HHSIZE'])**(1/2)

# Generate Real HHINCOME in 2018 dollars
raw['ERHHINCOME'] = raw['RHHINCOME']/raw['HHSIZE']

print(raw[['ISIZE', 'PERNUM', 'AGE', 'HHID', 'ERHHINCOME', 'HHSIZE']].head(5))
print(raw[['ISIZE', 'PERNUM', 'AGE', 'HHID', 'ERHHINCOME', 'HHSIZE']].describe())

    ISIZE  PERNUM  AGE  HHID  ERHHINCOME  HHSIZE
0     1.0       1   48   0.0    35,881.2     1.4
2     1.0       1   47   1.0    87,438.2     2.2
7     1.0       1   63   2.0    72,742.1     1.4
9     1.0       1   29   3.0    27,472.5     1.4
11    1.0       1   29   4.0    29,616.7     2.0
            ISIZE      PERNUM         AGE        HHID  ERHHINCOME      HHSIZE
count 2,763,219.0 2,763,219.0 2,763,219.0 2,763,219.0 2,763,219.0 2,763,219.0
mean          1.0         1.0        48.4 1,381,609.0    47,773.9         1.6
std           0.0         0.0        17.0   797,672.8    47,075.3         0.4
min           1.0         1.0         0.0         0.0   -52,725.6         1.0
25%           1.0         1.0        35.0   690,804.5    20,260.0         1.4
50%           1.0         1.0        46.0 1,381,609.0    37,183.8         1.4
75%           1.0         1.0        61.0 2,072,413.5    61,021.6         2.0
max           1.0         1.0        99.0 2,763,218.0 2,122,072.5         5.1


### **2.3. State price**

Prices in different states vary widely. According to the BEA, in 2018, for example, Alabama’s state price level for all items is 86.4. California was a third higher at 115.4. Thus, adjusting for price differences across states matters. We use the [BEA's Regional Price](https://www.bea.gov/data/prices-inflation/regional-price-parities-state-and-metro-area) Parities data from 2008 (the earliest year) to 2018 to adjust RHHINCOME and ERHHINCOME. 

In [ ]:
# Adjusting for state price using RPP
raw['RPPRHHINCOME'] = raw['RHHINCOME']/(raw['RPP']/100)
raw['RPPERHHINCOME'] = raw['ERHHINCOME']/(raw['RPP']/100)

# Generate cumulative weight and percentage
raw['CUMWTH'] = np.nan
raw['PERCENTH'] = np.nan

print(raw[['ERHHINCOME', 'RHHINCOME', 'RPPRHHINCOME', 'RPPERHHINCOME', 'RPP', 'CUMWTH', 'PERCENTH']].head(5))
raw.to_csv(input_path + 'processedRaw.csv', index=False)

    ERHHINCOME  RHHINCOME  RPPRHHINCOME  RPPERHHINCOME  RPP  CUMWTH  PERCENTH
0     35,881.2   50,743.7           nan            nan  nan     nan       nan
2     87,438.2  195,517.7           nan            nan  nan     nan       nan
7     72,742.1  102,872.9           nan            nan  nan     nan       nan
9     27,472.5   38,852.0           nan            nan  nan     nan       nan
11    29,616.7   59,233.3           nan            nan  nan     nan       nan


## **Section 3. getIncomeVis() Function**

After the 1977 demonstration, we use `getIncomeVis()` function to process data for samples from `year_start` to `year_end`, which yields HHINCOME from `year_start-1` to `year_end-1`. Following is syntax of `getIncomeVis()` function with its default parameters:

```
(DataFrame) getIncomeVis(incomeType = 'RHHINCOME',
                         k = 'decile',
                         year_start = 1977,
                         year_end = 2019, 
                         input_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/input/' + 'processedRaw.csv', 
                         output_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/output/',
                         toState = True,
                         provide_colorFrame = False,
                         colorFrame = pd.DataFrame(),
                         returnColor = False)
```

*   `incomeType(String)`: either `'HHINCOME'`, `'RHHINCOME'`, `'ERHHINCOME'`, `'RPPRHHINCOME'`, or `'RPPERHHINCOME'`.
*   `k(String)`: either `'decile'` or `'percentile'`. You must have `decile` and `percentile` folders in output directory to store output.
*   `year_start(int)`: starting year.
*   `year_end(int)`: ending year (unlike conventional index in Python, this year will also be included).
*   `input_path(String)`: file path of `processedRaw.csv` file.
*   `output_path(String)`: directory to output result.
*   `toState(bool)`: if `True`, output State graphs for the specified range (also output csv file for both Year and State graphs).
*   `provide_colorFrame(bool)`: where or not to use a precreated color frame.
*   `colorFrame(Pandas DataFrame)`: will be ignore if `provide_colorFrame = False`.
*   `returnColor(bool)`: if `True`, will only process year_start and return color frame of that year. 


In [ ]:
#@title **getIncomeVis() Implementation for Developer**
from contextlib import contextmanager
import os, sys
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try: yield
        finally: sys.stdout = old_stdout

import pandas as pd
import numpy as np
import json
from collections import OrderedDict
from google.colab import drive
with suppress_stdout(): drive.mount('/content/gdrive')
def getIncomeVis(incomeType = 'RHHINCOME',
                 k = 'decile', 
                 year_start = 1977,
                 year_end = 2019, 
                 input_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/input/' + 'processedRaw.csv',
                 output_path = 'gdrive/My Drive/Colab Notebooks/USIncomeVis/output/',  
                 toState = False,
                 provide_colorFrame = False, 
                 colorFrame = pd.DataFrame(), 
                 returnColor = False
                 ):
  
  # Display 1 decimal
  pd.options.display.float_format = '{:,.1f}'.format
  
  # Show all columns
  pd.set_option('display.max_columns', None)
  pd.set_option('display.max_rows', None)

  # Get preprocessed raw file
  raw = pd.read_csv(input_path)

  # Generate statefip
  statefip = list(set(raw.STATEFIP))

  for year in range(year_start, year_end+1):
    # Generate year_df dataframe
    year_df = raw[raw.YEAR == year]

    # Labels 
    label_list = ['']

    state_name = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
                  'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
                  'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
                  'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
                  'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
                  'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
                  'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
                  'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
                  'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
                  'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

    colors = ['#FF0000', '#FF0A00', '#FF1400', '#FF1E00', '#FF2800', '#FF3300', '#FF3D00',
              '#FF4700', '#FF5100', '#FF5B00', '#FF6600', '#FF7000', '#FF7A00', '#FF8400',
              '#FF8E00', '#FF9900', '#FFA300', '#FFAD00', '#FFB700', '#FFC100', '#FFCC00',
              '#FFD600', '#FFE000', '#FFEA00', '#FFF400', '#FFFF00', '#F4FF00', '#EAFF00',
              '#E0FF00', '#D6FF00', '#CCFF00', '#C1FF00', '#B7FF00', '#ADFF00', '#A3FF00',
              '#99FF00', '#8EFF00', '#84FF00', '#7AFF00', '#70FF00', '#66FF00', '#5BFF00',
              '#51FF00', '#47FF00', '#3DFF00', '#32FF00', '#28FF00', '#1EFF00', '#14FF00',
              '#0AFF00', '#00FF00']

    # Create state_Name and state_label dataframes
    notLabelList = list(set(state_name) - set(label_list))
    notLabelDict = dict.fromkeys(notLabelList , '')
    state_name = pd.DataFrame(data = state_name, index = statefip, columns = ['State'])
    state_label = state_name.replace(to_replace = notLabelDict)
    state_label = state_label.rename(columns={'State': 'Label'})

    # Create decile and percentile arrays
    decile = np.arange(0.05, 1.05, 0.1) # 10
    decile = np.insert(arr = decile, obj = 5, values = 0.5) # 11
    
    percentile = np.arange(0.02, 1, 0.01) # 98

	  # Create decile and percentile name
    decileName = ['5p', '15p', '25p', '35p', '45p', '50p', '55p', '65p', '75p', '85p', '95p']
    percentileName = ['2p', '3p', '4p', '5p', '6p', '7p', '8p', '9p', '10p', '11p',
                  		'12p', '13p', '14p', '15p', '16p', '17p', '18p', '19p', '20p',
                		  '21p', '22p', '23p', '24p', '25p', '26p', '27p', '28p', '29p',
                  		'30p', '31p', '32p', '33p', '34p', '35p', '36p', '37p', '38p',
                  		'39p', '40p', '41p', '42p', '43p', '44p', '45p', '46p', '47p',
                  		'48p', '49p', '50p', '51p', '52p', '53p', '54p', '55p', '56p',
                  		'57p', '58p', '59p', '60p', '61p', '62p', '63p', '64p', '65p',
                  		'66p', '67p', '68p', '69p', '70p', '71p', '72p', '73p', '74p',
                  		'75p', '76p', '77p', '78p', '79p', '80p', '81p', '82p', '83p',
                  		'84p', '85p', '86p', '87p', '88p', '89p', '90p', '91p', '92p',
                  		'93p', '94p', '95p', '96p', '97p', '98p', '99p']
                  		
    if (k == 'decile'):
      k_ile = decile
      kName = decileName
    elif (k == 'percentile'):
      k_ile = percentile
      kName = percentileName
    else: print("Invalid k")

    # Generate result grid, decile-column
    result = pd.DataFrame(data = None, index = kName, columns = statefip)

    # Iterate through each state
    c = 0
    for i in statefip:
      # Generate state dataframe
      state_df = year_df[year_df.STATEFIP == i]
      state_df = state_df.reset_index(drop = True)

      # Sort state dataframe by RHHINCOME
      state_df = state_df.sort_values(incomeType)
    
      # Calculate cumulated weight and Percentage
      state_df.CUMWTH = state_df.ASECWTH.cumsum()
      state_df.PERCENTH = state_df.CUMWTH/(state_df.ASECWTH.sum())

      # Calculate decile
      r = 0
      for _k_ile in k_ile:
        result.iloc[r,c] = state_df.loc[state_df['PERCENTH'] <= _k_ile, incomeType].max()
        r = r + 1
      c = c + 1

    # Transpose result table: column-decile
    result = result.T

    # Sort the result by median
    result = result.sort_values(by = ['50p'], ascending = True)

    colors = ['#FF0000', '#FF0A00', '#FF1400', '#FF1E00', '#FF2800', '#FF3300', '#FF3D00',
              '#FF4700', '#FF5100', '#FF5B00', '#FF6600', '#FF7000', '#FF7A00', '#FF8400',
              '#FF8E00', '#FF9900', '#FFA300', '#FFAD00', '#FFB700', '#FFC100', '#FFCC00',
              '#FFD600', '#FFE000', '#FFEA00', '#FFF400', '#FFFF00', '#F4FF00', '#EAFF00',
              '#E0FF00', '#D6FF00', '#CCFF00', '#C1FF00', '#B7FF00', '#ADFF00', '#A3FF00',
              '#99FF00', '#8EFF00', '#84FF00', '#7AFF00', '#70FF00', '#66FF00', '#5BFF00',
              '#51FF00', '#47FF00', '#3DFF00', '#32FF00', '#28FF00', '#1EFF00', '#14FF00',
              '#0AFF00', '#00FF00']
    colors = pd.DataFrame(colors, columns = ['Color'], index = statefip)
    
    if(not provide_colorFrame): colorFrame = pd.DataFrame(data = list(colors.Color), index = result.index, columns=['Color'])
    result = pd.concat([state_name, result, state_label, colorFrame], axis = 1)

    if (returnColor): return colorFrame

    # Compute state population and normalized state population
    pop = pd.DataFrame(index = statefip)
    pop['POP'] = year_df.groupby(['STATEFIP'])['ASECWT'].agg('sum')
    pop['NORMPOP'] = round(pop['POP']/(np.percentile(pop['POP'], 10)))

    # Replicate each state's dataline with its respective replication number
    for i in statefip:
      rep = pop.loc[i,'NORMPOP'] - 1
      rep = int(rep)
      line = pd.DataFrame(result.loc[i]).T
      line.loc[i, 'Label'] = ''
      for i in range(0,rep): result = pd.concat([result, line]) 

    # Sort the result by median
    result = result.sort_values(by = ['50p', 'State'], ascending = True)

    # Add the middle property
    result.reset_index(drop = True, inplace = True)

    result['Middle'] = np.nan
    counter = 0
    for state in result.State.drop_duplicates():
      temp = result[result.State == state]
      temp_size = len(temp.index)
      middle = (temp_size // 2)
      counter = counter + middle
      result.loc[counter, 'Middle'] = 1
      counter = counter - middle + temp_size

    # Output csv file if csv_output is True
    if (toState): result.to_csv(output_path + k + '/' + 'YEAR' + str(year-1) + '_' + incomeType + '.csv', index = False)
    
    # Convert dataframe to JSON
    result = result.to_json(orient = 'records')
    result = json.loads(result, object_pairs_hook = OrderedDict)

    # Make JSON format readable
    result = json.dumps(result, indent = 4, sort_keys = False)

    # Save JSON file -- y-1 adjusts sample year_df to HHINCOME year_df
    with open(output_path + k + '/' + 'YEAR' + str(year-1) + '_' + incomeType + '.js', 'w') as outfile:
      outfile.write('var data =')
      outfile.write(result)

  if (toState):
    if (k == 'decile'):
      column_df = ['5p', '15p', '25p', '35p', '45p', '50p', '55p', '65p', '75p','85p', '95p', 'Label']
      new_column = ['Year', '5p', '15p', '25p', '35p', '45p', '50p', '55p', '65p', '75p','85p', '95p', 'Label']
    elif (k == 'percentile'):
      column_df = ['2p', '3p', '4p', '5p', '6p', '7p', '8p', '9p', '10p', '11p',
                   '12p', '13p', '14p', '15p', '16p', '17p', '18p', '19p', '20p',
                   '21p', '22p', '23p', '24p', '25p', '26p', '27p', '28p', '29p',
                   '30p', '31p', '32p', '33p', '34p', '35p', '36p', '37p', '38p',
                   '39p', '40p', '41p', '42p', '43p', '44p', '45p', '46p', '47p',
                   '48p', '49p', '50p', '51p', '52p', '53p', '54p', '55p', '56p',
                   '57p', '58p', '59p', '60p', '61p', '62p', '63p', '64p', '65p',
                   '66p', '67p', '68p', '69p', '70p', '71p', '72p', '73p', '74p',
                   '75p', '76p', '77p', '78p', '79p', '80p', '81p', '82p', '83p',
                   '84p', '85p', '86p', '87p', '88p', '89p', '90p', '91p', '92p',
                   '93p', '94p', '95p', '96p', '97p', '98p', '99p', 'Label']
      new_column = ['Year', '2p', '3p', '4p', '5p', '6p', '7p', '8p', '9p', '10p', '11p',
                    '12p', '13p', '14p', '15p', '16p', '17p', '18p', '19p', '20p',
                    '21p', '22p', '23p', '24p', '25p', '26p', '27p', '28p', '29p',
                    '30p', '31p', '32p', '33p', '34p', '35p', '36p', '37p', '38p',
                    '39p', '40p', '41p', '42p', '43p', '44p', '45p', '46p', '47p',
                    '48p', '49p', '50p', '51p', '52p', '53p', '54p', '55p', '56p',
                    '57p', '58p', '59p', '60p', '61p', '62p', '63p', '64p', '65p',
                    '66p', '67p', '68p', '69p', '70p', '71p', '72p', '73p', '74p',
                    '75p', '76p', '77p', '78p', '79p', '80p', '81p', '82p', '83p',
                    '84p', '85p', '86p', '87p', '88p', '89p', '90p', '91p', '92p',
                    '93p', '94p', '95p', '96p', '97p', '98p', '99p', 'Label']
    else: print('Invalid k')

    for n in statefip:
      #Reformat the columns
      data_df = []
      index_df = [i for i in range(0, 43)]
      for i in range(year_start, year_end+1):
        df = pd.read_csv(output_path + k + '/' + 'YEAR' + str(year-1) + '_' + incomeType + '.csv')
        df.drop(columns=['Color', 'Middle'], inplace=True)
        t = df.loc[n].tolist()
        del t[0]
        data_df.append(t)

      state_df = pd.DataFrame(data_df,columns=column_df,index=index_df)
      state_df['Year'] = [i-1 for i in range(year_start, year_end + 1)]
      state_df['Label'] = ''

      #Convert to new Year formatted .csv files
      state_df = state_df.reindex(columns=new_column)
      state_df.to_csv(output_path + k + '/' + 'STATE' + str(n) + '_' + incomeType + '.csv', index=False)
  
      # Convert dataframe to JSON
      state_df = state_df.to_json(orient = 'records')
      state_df = json.loads(state_df, object_pairs_hook = OrderedDict)

      # Make JSON format readable
      state_df = json.dumps(state_df, indent = 4, sort_keys = False)

      # Save JSON file -- y-1 adjusts sample year to HHINCOME year
      with open(output_path + k + '/' + 'STATE' + str(n) + '_' + incomeType + '.js', 'w') as outfile:
        outfile.write("var data =")
        outfile.write(state_df)

In [ ]:
# Decile
colorRHHINCOME1977 = getIncomeVis(incomeType = 'RHHINCOME', returnColor = True)
getIncomeVis(incomeType = 'RHHINCOME', provide_colorFrame = True, colorFrame = colorRHHINCOME1977, toState = True)

colorERHHINCOME1977 = getIncomeVis(incomeType = 'ERHHINCOME', returnColor = True)
getIncomeVis(incomeType = 'ERHHINCOME', provide_colorFrame = True, colorFrame = colorERHHINCOME1977, toState = True)

colorRPPRHHINCOME1977 = getIncomeVis(incomeType = 'RPPRHHINCOME', returnColor = True)
getIncomeVis(incomeType = 'RPPRHHINCOME', provide_colorFrame = True, colorFrame = colorRPPRHHINCOME1977, toState = True)

colorRPPERHHINCOME1977 = getIncomeVis(incomeType = 'RPPERHHINCOME', returnColor = True)
getIncomeVis(incomeType = 'RPPERHHINCOME', provide_colorFrame = True, colorFrame = colorRPPERHHINCOME1977, toState = True)

colorRHHINCOME1977 = getIncomeVis(incomeType = 'HHINCOME', returnColor = True)
getIncomeVis(incomeType = 'HHINCOME', provide_colorFrame = True, colorFrame = colorRHHINCOME1977, toState = True)

# Percentile
colorRHHINCOME1977 = getIncomeVis(incomeType = 'RHHINCOME', returnColor = True, k = 'percentile')
getIncomeVis(incomeType = 'RHHINCOME', provide_colorFrame = True, colorFrame = colorRHHINCOME1977, toState = True, k = 'percentile')

colorERHHINCOME1977 = getIncomeVis(incomeType = 'ERHHINCOME', returnColor = True, k = 'percentile')
getIncomeVis(incomeType = 'ERHHINCOME', provide_colorFrame = True, colorFrame = colorERHHINCOME1977, toState = True, k = 'percentile')

colorRPPRHHINCOME1977 = getIncomeVis(incomeType = 'RPPRHHINCOME', returnColor = True, k = 'percentile')
getIncomeVis(incomeType = 'RPPRHHINCOME', provide_colorFrame = True, colorFrame = colorRPPRHHINCOME1977, toState = True, k = 'percentile')

colorRPPERHHINCOME1977 = getIncomeVis(incomeType = 'RPPERHHINCOME', returnColor = True, k = 'percentile')
getIncomeVis(incomeType = 'RPPERHHINCOME', provide_colorFrame = True, colorFrame = colorRPPERHHINCOME1977, toState = True, k = 'percentile')

colorRHHINCOME1977 = getIncomeVis(incomeType = 'HHINCOME', returnColor = True, k = 'percentile')
getIncomeVis(incomeType = 'HHINCOME', provide_colorFrame = True, colorFrame = colorRHHINCOME1977, toState = True, k = 'percentile')

### **3.1. HHINCOME**

Our goal is to prepare the data for rendering our final graph in JavaScript format. We segment HHINCOME into percentiles from 5 to 95 in steps of 10 and also the median. The code below can also produce increments of 1 percentile,  but then amCharts (our visualizer) is extremely slow.
Since we allow the user to control the states being labeled, we do not label any of them. 
We also create CUMWTH (cummulated household weight) and PERCENT (percentile for each household) as a way to rank income of each household

For the purpose of demostration and explanation, we walk through 1977 decile below to illustrate the mechanism behind the analysis. We do so by breaking the first for loop so that we render 1 graph for 1 state at a time. After that, we generate a result grid, which have the k-iles for each state at a year. 

In [ ]:
  # Generate year dataframe -- can enter any year from 1977 to 2019
  y = 1977
  year = raw[raw.YEAR == y]

  # Create decile and percentile arrays
  decile = np.arange(0.05, 1.05, 0.1) # 10
  decile = np.insert(arr = decile, obj = 5, values = 0.5) # 11
  percentile = np.arange(0.02, 1, 0.01) # 98

  # Create decile and percentile name
  decileName = ['5p', '15p', '25p', '35p', '45p', '50p', '55p', '65p', '75p', '85p', '95p']
  percentileName = ['2p', '3p', '4p', '5p', '6p', '7p', '8p', '9p', '10p', '11p',
                    '12p', '13p', '14p', '15p', '16p', '17p', '18p', '19p', '20p',
                    '21p', '22p', '23p', '24p', '25p', '26p', '27p', '28p', '29p',
                    '30p', '31p', '32p', '33p', '34p', '35p', '36p', '37p', '38p',
                    '39p', '40p', '41p', '42p', '43p', '44p', '45p', '46p', '47p',
                    '48p', '49p', '50p', '51p', '52p', '53p', '54p', '55p', '56p',
                    '57p', '58p', '59p', '60p', '61p', '62p', '63p', '64p', '65p',
                    '66p', '67p', '68p', '69p', '70p', '71p', '72p', '73p', '74p',
                    '75p', '76p', '77p', '78p', '79p', '80p', '81p', '82p', '83p',
                    '84p', '85p', '86p', '87p', '88p', '89p', '90p', '91p', '92p',
                    '93p', '94p', '95p', '96p', '97p', '98p', '99p']

  # Labels 
  label_list = ['']

  # Create state_Name and state_label dataframes
  notLabelList = list(set(state_name) - set(label_list))
  notLabelDict = dict.fromkeys(notLabelList , '')
  state_name = pd.DataFrame(data = state_name, index = statefip, columns = ['State'])
  state_label = state_name.replace(to_replace = notLabelDict)
  state_label = state_label.rename(columns={'State': 'Label'})

  # Choose k: decile (11) or percentile (98)
  k = 'decile'
  if (k == 'decile'):
    k_ile = decile
    kName = decileName

  if (k == 'percentile'):
    k_ile = percentile
    kName = percentileName

  # Generate result grid, decile-column
  result = pd.DataFrame(data = None, index = kName, columns = statefip)
  print(result[[1, 2, 4]].head(5))

       1    2    4
5p   NaN  NaN  NaN
15p  NaN  NaN  NaN
25p  NaN  NaN  NaN
35p  NaN  NaN  NaN
45p  NaN  NaN  NaN


For every state, we sort the adjusted household income ascendingly. We then calculate the household cumulated weight (CUMWTH) and the household percentile (PERCENTH). For household $j$th in the RHHINCOME-sorted dataframe with $k$ households:

$$CUMWTH_j = \sum_{i=0}^{j} ASECWTH_i; PERCENTH_j = \frac{CUMWTH_j}{\sum_{i=0}^{k} ASECWTH_i}$$

To calculate RHHINCOME at each k_ile $k$ for state $s$, we select a series of RHHINCOME that has percentile under or equal to $k$ (set S):
$$S_k = \{RHHINCOME_s | PERCENTH_s \leq k \} $$

RHHINCOME at k_ile $k$ for state s is the maximum value of set $S$: 

$$ RHHINCOME_{k, s} = max(S_k) $$

In [ ]:
  # Iterate through each state
  c = 0
  for i in statefip:
    # Generate state dataframe
    state = year[year.STATEFIP == i]
    state = state.reset_index(drop = True)

    # Sort state dataframe by RHHINCOME
    state = state.sort_values('RHHINCOME')
    
    # Calculate cumulated weight and Percentage
    state.CUMWTH = state.ASECWTH.cumsum()
    state.PERCENTH = state.CUMWTH/(state.ASECWTH.sum())
    
    # Calculate decile
    r = 0
    for _k_ile in k_ile:
      result.iloc[r,c] = state.loc[state['PERCENTH'] <= _k_ile, 'RHHINCOME'].max()
      r = r + 1
    c = c + 1

  # Transpose result table: column-decile
  result = result.T

  # Sort the result by median
  result = result.sort_values(by = ['50p'], ascending = True)
  colors1977 = pd.DataFrame(data = list(colors.Color), index = result.index, columns=['Color'])
  result = pd.concat([state_name, result, state_label, colors1977], axis = 1)
  print(result.head(5))

        State       5p      15p      25p      35p      45p      50p      55p  \
1     Alabama  8,039.6 15,201.0 23,651.0 32,224.5 42,721.8 48,272.7 52,950.0   
2      Alaska 12,178.5 34,029.2 48,537.5 66,187.4 81,631.2 88,964.7 98,839.9   
4     Arizona  8,229.3 18,183.9 26,916.2 38,318.1 47,434.3 52,839.6 57,362.4   
5    Arkansas  8,458.8 13,237.5 19,891.5 26,475.0 34,417.5 37,422.4 41,106.8   
6  California 12,160.8 20,015.1 30,887.5 40,330.2 51,855.7 57,450.7 64,051.8   

        65p       75p       85p       95p Label    Color  
1  63,981.2  77,262.8  94,859.8 131,933.6        #FF5100  
2 117,001.7 139,589.3 174,734.8 223,122.3        #00FF00  
4  68,609.9  82,513.7  98,853.1 152,897.4        #FFCC00  
5  49,742.1  62,414.8  76,243.5 115,519.1        #FF0000  
6  77,218.7  92,212.3 114,164.5 166,192.2        #A3FF00  


### **3.2. Population**

We compute an estimate of the number of households in the population (POP) and then normalize it for each state in a new dataframe (NORMPOP). For state $s$ that has $k$ households in year $y$, we could make each state's population relative to the smallest state, like this:

$$POP_s = \sum_{i=0}^{k} ASECWTH_i; NORMPOP_s = \frac{POP_s}{min(POP_s)}$$ 


However, this turns out to produce an ugly graph (with extremely thin slices for the smallest states) and takes a long time to render in a browser. To improve visibility and rendering, we normalize to the 10th percentile which makes about 1/4 of the states have a width of 1 in the graph. California (the most populous state) has a width 34 times the smallest states (in actuality it should be 68 times).

We replicate each state dataline with its relative population. Sort the result dataframe by the median to get the 3D visualization to plot the states in ascending order of household income. 

In [ ]:
  # Compute state population and normalized state population
  pop = pd.DataFrame(index = statefip)
  pop['POP'] = year.groupby(['STATEFIP'])['ASECWT'].agg('sum')
  pop['NORMPOP'] = round(pop['POP']/(pop['POP'].min()))

  # Normalization
  pop['NORMPOP'] = round(pop['POP']/(np.percentile(pop['POP'],10)))
  print(pop.head())
  
  # Replicate each state's dataline with its respective replication number
  for i in statefip:
    rep = pop.loc[i,'NORMPOP'] - 1
    rep = int(rep)
    line = pd.DataFrame(result.loc[i]).T
    line.loc[i, 'Label'] = ''
    for i in range(0,rep): result = pd.concat([result, line])

  # Sort the result by median
  result = result.sort_values(by = ['50p', 'State'], ascending = True)
  result.reset_index(drop = True, inplace = True)

  # Enables the user to add a label to the middle slice in the graph in AmChart
  result['Middle'] = np.nan
  counter = 0
  for state in result.State.drop_duplicates():
    temp = result[result.State == state]
    temp_size = len(temp.index)
    middle = (temp_size // 2)
    counter = counter + middle
    result.loc[counter, 'Middle'] = 1
    counter = counter - middle + temp_size
  print(result.head(5))

          POP  NORMPOP
1 1,301,714.0      6.0
2   116,622.3      1.0
4   789,233.0      3.0
5   687,999.6      3.0
6 7,934,076.9     34.0


### **3.3. Output to JSON**

In [ ]:
  # Convert dataframe to JSON, the required input for amCharts to generate the visualization.
  result = result.to_json(orient = 'records')
  result = json.loads(result, object_pairs_hook = OrderedDict)

  # Make JSON format readable
  result = json.dumps(result, indent = 4, sort_keys = False)

  # Insert result to HTML environment
  with open(out_path + k + '/' + str(y-1) + '_d_' + 'RHHHHINCOME' + '.js', 'w') as outfile:
    outfile.write('var data =')
    outfile.write(result)

## **Section 6. Conclusion**

We now have four variables sorted by median for each state with population determining how many times we repeat the rows for each state:

*   RHHINCOME: real household income
*   ERHHINCOME: real household income per (equivalized) person)
*   RHHINCOMERPP: real household income adjusted for state prices
*   ERHHINCOMERPP: real household income per (equivalized) person) adjusted for state prices 

With the data prepared, we turn to amCharts to render the graph.

**References:**

[1] Sarah Flood, Miriam King, Renae Rodgers, Steven Ruggles and J. Robert Warren. Integrated Public Use Microdata Series, Current Population Survey: Version 6.0 [dataset]. Minneapolis, MN: IPUMS, 2018.
https://doi.org/10.18128/D030.V6.0

[2] https://cps.ipums.org/cps-action/variables/statefip#comparability_section 

[3] https://cps.ipums.org/cps/three_eighths.shtml).

[4] Johnson, D., Smeeding T., and Boyle Torrey, B. "Economic inequality through the prisms
of income and consumption," *Monthly Labor Review* (Apr 2005), https://www.bls.gov/opub/mlr/2005/04/art2full.pdf.

[5] Color generator: https://www.strangeplanet.fr/work/gradient-generator/index.php

[6] AmChart documentation:  https://docs.amcharts.com/3/javascriptcharts/AmGraph

[7] Jack Blundell's graph: https://jackblun.github.io/Globalinc/html/fig_1980.html
